In [1]:
import os
import numpy as np
import helper as hp
import pandas as pd
import multiprocessing as mp
from configparser import ConfigParser

from evostrat.init_mlp import MLP
from evostrat.evolution_strategy import EvolutionStrategy

from kinetics.jacobian_solver import check_jacobian

#Parse arguments from configfile
configs = ConfigParser()
configs.read('configfile.ini')

n_samples = int(configs['MLP']['n_samples'])

lnminkm = float(configs['CONSTRAINTS']['min_km'])
lnmaxkm = float(configs['CONSTRAINTS']['max_km'])

repeats = int(configs['EVOSTRAT']['repeats'])
save_step = int(configs['EVOSTRAT']['save_step'])
generations = int(configs['EVOSTRAT']['generations'])
pop_size = int(configs['EVOSTRAT']['pop_size'])
noise = float(configs['EVOSTRAT']['noise'])
lr = float(configs['EVOSTRAT']['lr'])
decay = float(configs['EVOSTRAT']['decay'])
ss_idx = int(configs['EVOSTRAT']['ss_idx'])
n_threads = int(configs['EVOSTRAT']['n_threads'])

output_path = configs['PATHS']['output_path']
met_model = configs['PATHS']['met_model']
names_km = hp.load_pkl(f'models/{met_model}/parameter_names_km_fdp1.pkl')
regulation_file = configs['PATHS']['regulation_file']
steady_states_file = configs['PATHS']['steady_states_file']
path_regulation_file = f'models/{met_model}/{regulation_file}'
path_steady_states = f'models/{met_model}/{steady_states_file}'

reward_flag = int(configs['REWARDS']['reward_flag'])
eig_partition = float(configs['REWARDS']['eig_partition'])
n_consider = int(configs['REWARDS']['n_consider'])

pf_flag = int(configs['PARAMETER_FIXING']['pf_flag'])

# Call solvers from SKimPy
chk_jcbn = check_jacobian()



In [2]:
# declare reward functions
def reward_func(weights):
    """
    evaluate reward for a set of generated kinetic parameter sets
    :param gen_params: agent generated kinetic parameter sets
    :return: reward
    """

    global calc_eig
    global n_samples
    global eig_partition
    global n_threads
    global reward_flag
    global n_consider

    def calc_eig(gen_param):
        chk_jcbn._prepare_parameters(gen_param, k_names)
        max_eig = chk_jcbn.calc_eigenvalues_recal_vmax()
        return max_eig
    pool = mp.Pool(n_threads)

    mlp.generator.set_weights(weights)
    gen_params = mlp.sample_parameters()
    gen_params = [[params] for params in gen_params]

    max_eig = pool.map(calc_eig, gen_params)
    max_eig = np.array([this_eig for eig in max_eig for this_eig in eig])

    if reward_flag == 0:
        max_neg_eig = np.min(max_eig)
        if max_neg_eig > eig_partition:
            this_reward = 0.01 / (1 + np.exp(max_neg_eig - eig_partition))
        else:
            this_reward = len(np.where(max_eig <= eig_partition)[0]) / n_samples
        pool.close()
    elif reward_flag == 1:
        max_eig.sort()
        considered_avg = sum(max_eig[:n_consider]) / n_consider
        this_reward = np.exp(-0.1 * considered_avg) / 2
        pool.close()

    return this_reward

In [3]:
# Integrate regulation data
regulation_table = pd.read_csv(path_regulation_file)
chk_jcbn._load_regulation_ktmodels(met_model, regulation_table, "fdp1") ## Load kinetic and thermodynamic data

steady_states = pd.read_csv(path_steady_states, header=0, index_col=0).iloc[ss_idx, 0:]
chk_jcbn._load_ssprofile(steady_states)  ## Integrate steady state information


2024-03-01 11:03:03,710 - thermomodel_new - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2024-03-01 11:03:07,085 - Unnamed - WARNING - Non integer stoichiometries found ['CYTBO3_4pp', 'LMPD_biomass_c_1_420', 'CYTBDpp'] change to integer for linear dependencies


These parameters are omitted due to KeyError: 
['hill_coefficient_ATPM', 'k1_bwd_ACALDtex', 'k1_bwd_ACtex', 'k1_bwd_AKGtex', 'k1_bwd_CO2tex', 'k1_bwd_D-LACtex', 'k1_bwd_ETOHtex', 'k1_bwd_FORtex', 'k1_bwd_GLCtex', 'k1_bwd_GLUtex', 'k1_bwd_Htex', 'k1_bwd_MALtex', 'k1_bwd_NH4tex', 'k1_bwd_O2tex', 'k1_bwd_PHEtex', 'k1_bwd_PYRtex', 'k1_bwd_SERtex', 'k1_bwd_SUCCtex', 'k1_bwd_TRPtex', 'k1_bwd_TYRtex', 'k1_fwd_ACALDtex', 'k1_fwd_ACtex', 'k1_fwd_AKGtex', 'k1_fwd_CO2tex', 'k1_fwd_D-LACtex', 'k1_fwd_ETOHtex', 'k1_fwd_FORtex', 'k1_fwd_GLCtex', 'k1_fwd_GLUtex', 'k1_fwd_Htex', 'k1_fwd_MALtex', 'k1_fwd_NH4tex', 'k1_fwd_O2tex', 'k1_fwd_PHEtex', 'k1_fwd_PYRtex', 'k1_fwd_SERtex', 'k1_fwd_SUCCtex', 'k1_fwd_TRPtex', 'k1_fwd_TYRtex', 'k2_bwd_ACALDtex', 'k2_bwd_ACtex', 'k2_bwd_AKGtex', 'k2_bwd_CO2tex', 'k2_bwd_D-LACtex', 'k2_bwd_ETOHtex', 'k2_bwd_FORtex', 'k2_bwd_GLCtex', 'k2_bwd_GLUtex', 'k2_bwd_Htex', 'k2_bwd_MALtex', 'k2_bwd_NH4tex', 'k2_bwd_O2tex', 'k2_bwd_PHEtex', 'k2_bwd_PYRtex', 'k2_bwd_SERtex', 'k2_

2024-03-01 11:03:10,655 - Unnamed - WARNING - Non integer stoichiometries found ['LMPD_biomass_c_1_420', 'CYTBDpp', 'CYTBO3_4pp'] change to integer for linear dependencies


In [4]:
k_names = set()
for k,v in chk_jcbn.kmodel.parameters.items():
    if k.startswith("km_"):
        k_names.add(k)
    elif ("activation" in k) or ("inhibition" in k):
        k_names.add(k)
print(f"The number of k for generation is {len(k_names)}")

The number of k for generation is 406


In [5]:

for rep in range(repeats):
    print(f"repeat {rep}")
    cond_class = 1
    # Call neural network agent
    mlp = MLP(cond_class, lnminkm, lnmaxkm, n_samples, k_names, len(k_names), param_fixing=pf_flag)
    '''
    ## Snippet to start from previously saved weights 
    load_weights = hp.load_pkl('/path/to/weights')
    mlp.generator.set_weights(load_weights)
    '''
    init_dict = mlp.generator.get_weights()

    this_savepath = f'{output_path}/repeat_{rep}/'
    os.makedirs(this_savepath, exist_ok=True)

    es = EvolutionStrategy(mlp.generator.get_weights(),
                           reward_func, this_savepath,
                           population_size=pop_size,
                           sigma=noise,  # noise std deviation
                           learning_rate=lr,
                           decay=decay,
                           num_threads=1)

    rewards = es.run(generations, print_step=save_step)
    hp.save_pkl(f'{this_savepath}/rewards', rewards)



repeat 0
99


starting
   km_substrate1_ADK1  km_substrate21_LMPD_biomass_c_1_420  km_substrate2_GND  \
0               False                                False              False   

   km_product1_COLIPAabctex  km_product1_PFL  km_substrate1_TYRTA  \
0                     False            False                False   

   km_substrate1_FDH4pp  km_substrate2_PSCVT  km_product3_PPCK  \
0                 False                False             False   

   km_product1_CYTBO3_4pp  ...  km_substrate1_ATPM  km_substrate3_SUCOAS  \
0                   False  ...               False                 False   

   km_product1_NADH16pp  km_substrate2_G6PDH2r  km_product3_GLCabcpp  \
0                 False                  False                 False   

   km_product1_PFK  km_product2_NADTRHD  km_product1_PSCVT  \
0            False                False              False   

   km_substrate_GLCt2pp  km_substrate1_CYTBDpp  
0                 False                  False  

[1 rows x 406 columns]   km_substr

/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy

   km_substrate1_ADK1  km_substrate21_LMPD_biomass_c_1_420  km_substrate2_GND  \
0               False                                False              False   

   km_product1_COLIPAabctex  km_product1_PFL  km_substrate1_TYRTA  \
0                     False            False                False   

   km_substrate1_FDH4pp  km_substrate2_PSCVT  km_product3_PPCK  \
0                 False                False             False   

   km_product1_CYTBO3_4pp  ...  km_substrate1_ATPM  km_substrate3_SUCOAS  \
0                   False  ...               False                 False   

   km_product1_NADH16pp  km_substrate2_G6PDH2r  km_product3_GLCabcpp  \
0                 False                  False                 False   

   km_product1_PFK  km_product2_NADTRHD  km_product1_PSCVT  \
0            False                False              False   

   km_substrate_GLCt2pp  km_substrate1_CYTBDpp  
0                 False                  False  

[1 rows x 406 columns]   km_substrate1_ADK1

/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less


...

   km_substrate1_ADK1  km_substrate21_LMPD_biomass_c_1_420  km_substrate2_GND  \
0               False                                False              False   

   km_product1_COLIPAabctex  km_product1_PFL  km_substrate1_TYRTA  \
0                     False            False                False   

   km_substrate1_FDH4pp  km_substrate2_PSCVT  km_product3_PPCK  \
0                 False                False             False   

   km_product1_CYTBO3_4pp  ...  km_substrate1_ATPM  km_substrate3_SUCOAS  \
0                   False  ...               False                 False   

   km_product1_NADH16pp  km_substrate2_G6PDH2r  km_product3_GLCabcpp  \
0                 False                  False                 False   

   km_product1_PFK  km_product2_NADTRHD  km_product1_PSCVT  \
0            False                False              False   

   km_substrate_GLCt2pp  km_substrate1_CYTBDpp  
0                 False                  False  

[1 rows x 406 columns]


/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less


   km_substrate1_ADK1  km_substrate21_LMPD_biomass_c_1_420  km_substrate2_GND  \
0               False                                False              False   

   km_product1_COLIPAabctex  km_product1_PFL  km_substrate1_TYRTA  \
0                     False            False                False   

   km_substrate1_FDH4pp  km_substrate2_PSCVT  km_product3_PPCK  \
0                 False                False             False   

   km_product1_CYTBO3_4pp  ...  km_substrate1_ATPM  km_substrate3_SUCOAS  \
0                   False  ...               False                 False   

   km_product1_NADH16pp  km_substrate2_G6PDH2r  km_product3_GLCabcpp  \
0                 False                  False                 False   

   km_product1_PFK  km_product2_NADTRHD  km_product1_PSCVT  \
0            False                False              False   

   km_substrate_GLCt2pp  km_substrate1_CYTBDpp  
0                 False                  False  

[1 rows x 406 columns]
   km_substrate1_ADK

/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less
/renaissance/skimpy/sampling/flux_parameter_function.py:66 RuntimeWarning: invalid value encountered in less


ValueError: array must not contain infs or NaNs

In [6]:
a = np.array([np.inf, 0, np.inf])
sum(np.isinf(a))

2